In [ ]:
from pygame.locals import KEYDOWN, K_ESCAPE, K_q ,K_x, K_w
import pygame
from pygame.locals import Color, QUIT, MOUSEBUTTONDOWN, USEREVENT, USEREVENT
import cv2
import sys
# import tensorflow as tf
import numpy as np
import time
import random
import os

# 視窗大小
WIN_WIDTH = 1280
WIN_HEIGHT = 720
# 鳥鳥大小
IMAGEWIDTH = 50
IMAGEHEIGHT = 50

# 開始結束畫面
# STAT_FONT = pygame.font.SysFont("comicsans", 50)
# END_FONT = pygame.font.SysFont("comicsans", 70)



face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier('haarcascade_smile.xml') 





def detect_smile(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    faces = face_cascade.detectMultiScale(gray, 1.3, 5) 
    for (x, y, w, h) in faces: 
#         cv2.rectangle(frame, (x, y), ((x + w), (y + h)), (255, 0, 0), 2) 
        roi_gray = gray[y:y + h, x:x + w] 
        roi_color = frame[y:y + h, x:x + w] 
        smiles = smile_cascade.detectMultiScale(roi_gray, 1.8, 20)
#         for (sx, sy, sw, sh) in smiles: 
#             cv2.rectangle(roi_color, (sx, sy), ((sx + sw), (sy + sh)), (0, 0, 255), 2)
        
        if len(smiles):
            return 1
        else:
            return 0





class Pipe():
    """
    represents a pipe object
    """
    GAP = 250
    VEL = 10

    def __init__(self, x):
        """
        initialize pipe object
        :param x: int
        :param y: int
        :return" None
        """
        self.x = x
        self.height = 0

        # where the top and bottom of the pipe is
        self.top = 0
        self.bottom = 0
        
        pipe_img = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","pipe.png")))
        self.PIPE_TOP = pygame.transform.flip(pipe_img, False, True)
        self.PIPE_BOTTOM = pipe_img
        
#         self.rect = self.image.get_rect()
#         self.rect.topleft = (self.x, self.y)

        self.passed = False

        self.set_height()

    def set_height(self):
        """
        set the height of the pipe, from the top of the screen
        :return: None
        """
        self.height = random.randrange(25, 300)
        self.top = self.height - self.PIPE_TOP.get_height()
        self.bottom = self.height + self.GAP

    def move(self):
        """
        move pipe based on vel
        :return: None
        """
        self.x -= self.VEL

    def draw(self, win):
        """
        draw both the top and bottom of the pipe
        :param win: pygame window/surface
        :return: None
        """
        # draw top
        win.blit(self.PIPE_TOP, (self.x, self.top))
        # draw bottom
        win.blit(self.PIPE_BOTTOM, (self.x, self.bottom))
        
        
        
        
        
# class Mosquito(pygame.sprite.Sprite):
#     def init(self):
#         super().init()
#         pygame.sprite.Sprite.init(self)
#         self.x = random.randrange(0, 640-IMAGEWIDTH)
#         self.y = random.randrange(-20, 0)
#         self.raw_image = pygame.image.load('UFO.png')
#         self.image = pygame.transform.scale(self.raw_image, (IMAGEWIDTH, IMAGEHEIGHT))
#         self.rect = self.image.get_rect()
#         self.rect.topleft = (self.x, self.y)
#         self.width = IMAGEWIDTH
#         self.height = IMAGEHEIGHT
#         self.widow_width = WINDOW_WIDTH
#         self.window_height = WINDOW_HEIGHT

#     def draw(self, win):
#         win.blit(self.image, self.rect)
#     def move(self):
#         self.x += random.randrange(-10, 10)
#         self.y += 10
#         self.rect.topleft = (self.x, self.y)
   
        
        
        
bird_images = [pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird" + str(x) + ".png"))) for x in range(1,4)]        


class Bird:
    """
    Bird class representing the flappy bird
    """
    MAX_ROTATION = 25
#     bird_images = pygame.transform.scale2x(pygame.image.load(os.path.join("imgs","bird1.png")))
    IMGS = bird_images
    ROT_VEL = 20
    ANIMATION_TIME = 5
    VEL_drop = 5
    VEL_jump = 25 # 13

    def __init__(self, x, y):
        """
        Initialize the object
        :param x: starting x pos (int)
        :param y: starting y pos (int)
        :return: None
        """
        self.x = x
        self.y = y
        self.img_width = IMAGEWIDTH
        self.img_height = IMAGEHEIGHT
        self.tilt = 0  # degrees to tilt
        self.tick_count = 0
        self.height = self.y
        self.img_count = 0
        self.img = self.IMGS[0]
#         self.rect = self.image.get_rect()
        
    def draw(self, win):
        """
        draw the bird
        :param win: pygame window or surface
        :return: None
        """
        self.img_count += 1

        # For animation of bird, loop through three images
        if self.img_count <= self.ANIMATION_TIME:
            self.img = self.IMGS[0]
        elif self.img_count <= self.ANIMATION_TIME*2:
            self.img = self.IMGS[1]
        elif self.img_count <= self.ANIMATION_TIME*3:
            self.img = self.IMGS[2]
        elif self.img_count <= self.ANIMATION_TIME*4:
            self.img = self.IMGS[1]
        elif self.img_count == self.ANIMATION_TIME*4 + 1:
            self.img = self.IMGS[0]
            self.img_count = 0

        # so when bird is nose diving it isn't flapping
        if self.tilt <= -80:
            self.img = self.IMGS[1]
            self.img_count = self.ANIMATION_TIME*2


        # tilt the bird
        blitRotateCenter(win, self.img, (self.x, self.y), self.tilt)

        # win.blit(self.IMGS, (self.x, self.y))
    
    
    def drop(self):
        self.y += self.VEL_drop
        
    def jump(self):
        self.y -= self.VEL_jump

        
def blitRotateCenter(surf, image, topleft, angle):
    """
    Rotate a surface and blit it to the window
    :param surf: the surface to blit to
    :param image: the image surface to rotate
    :param topLeft: the top left position of the image
    :param angle: a float value for angle
    :return: None
    """
    rotated_image = pygame.transform.rotate(image, angle)
    new_rect = rotated_image.get_rect(center = image.get_rect(topleft = topleft).center)

    surf.blit(rotated_image, new_rect.topleft)

def check_collision(pipes):
    for pipe in pipes:
        if bird_rect.colliderect(pipe):
            return False

    if bird_rect.top <= -100 or bird_rect.bottom >= 900:
        return False

    return True


# def score_display(game_state):
#     if game_state == 'main_game':
#         score_surface = game_font.render(str(int(score)),True,(255,255,255))
#         score_rect = score_surface.get_rect(center = (288,100))
#         WIN.blit(score_surface,score_rect)
#     if game_state == 'game_over':
#         score_surface = game_font.render(f'Score: {int(score)}' ,True,(255,255,255))
#         score_rect = score_surface.get_rect(center = (288,100))
#         WIN.blit(score_surface,score_rect)

#         high_score_surface = game_font.render(f'High score: {int(high_score)}',True,(255,255,255))
#         high_score_rect = high_score_surface.get_rect(center = (288,850))
#         WIN.blit(high_score_surface,high_score_rect)

def update_score(score, high_score):
    if score > high_score:
        high_score = score
    return high_score

camera = cv2.VideoCapture(0)
camera_x, camera_y = (WIN_WIDTH, WIN_HEIGHT)     # Set camera resolution [1]=1280,720
camera.set(cv2.CAP_PROP_FRAME_WIDTH , camera_x)
camera.set(cv2.CAP_PROP_FRAME_HEIGHT, camera_y)





pygame.init()
WIN = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
game_font = pygame.font.Font('04B_19.ttf',40)
# Game Variables
gravity = 0.25
bird_movement = 0
score = 0
high_score = 0


def main():
    

    
    try:
        pygame.init()
        WIN = pygame.display.set_mode((WIN_WIDTH, WIN_HEIGHT))
        pygame.display.set_caption("Flappy Bird")
        pipe_move_event = USEREVENT+1
        pygame.time.set_timer(pipe_move_event, 100) # 設定每300毫秒更新一次

                    
        pipe1 = Pipe(WIN_WIDTH)
        pipe2 = Pipe(WIN_WIDTH/2)
        bird = Bird(10,240)
       
# WIN_WIDTH = 1280
# WIN_HEIGHT = 720   

        game_over_surface = pygame.transform.scale2x(pygame.image.load('assets/sprites/message.png').convert_alpha())
        game_over_rect = game_over_surface.get_rect(center = (WIN_WIDTH/2,WIN_HEIGHT/2))
        score_sound_countdown = 100
        clock = pygame.time.Clock()
        high_score = 0
        score = 0  
        game_active = True
        
        
        running = True
        start = True
        
        while running:
            
            while start:
                ret, frame = camera.read()
                WIN.fill([0, 0, 0])
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.flip(frame, 1) # 左右對調
                sm = detect_smile(frame)
                frame = frame.swapaxes(0, 1)
                pygame.surfarray.blit_array(WIN, frame)
                WIN.blit(game_over_surface,game_over_rect)
                
                for event in pygame.event.get():                
                    if event.type == KEYDOWN:
                        if event.key == K_ESCAPE or event.key == K_q:
                            sys.exit(0)
                            camera.release()
                        if event.key == K_w:
                            running = True
                            start  = False
                            score = 0
                
                
                pygame.display.update()
                
            while running:

                ret, frame = camera.read()
                WIN.fill([0, 0, 0])
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame = cv2.flip(frame, 1) # 左右對調
                sm = detect_smile(frame)
                frame = frame.swapaxes(0, 1)
                pygame.surfarray.blit_array(WIN, frame)

                pipe1.draw(WIN)
                pipe2.draw(WIN)
                bird.draw(WIN)

                for event in pygame.event.get():


                    if event.type == pygame.QUIT:
                        sys.exit(0)
                        camera.release()

                    if event.type == KEYDOWN:
                        if event.key == K_x:
                            bird.jump()

                        if event.key == K_q:

                            sys.exit(0)
                            camera.release()

                    if event.type == pipe_move_event:
                        pipe1.move()
                        pipe2.move()
                        bird.drop()

                        # 水管輪動
                        if pipe1.x==0:
                            pipe1 = Pipe(WIN_WIDTH)                      
                            pipe1.draw(WIN)                        
                        if pipe2.x==0:
                            pipe2 = Pipe(WIN_WIDTH)
                            pipe2.draw(WIN)
                            



                        # 水管碰撞 

                        if pipe1.x == bird.x + bird.img_width:
                            if pipe1.height >= bird.y:
                                
                                print('pipe1_height:', pipe1.height)
                                pipe1 = Pipe(WIN_WIDTH)
                                pipe2 = Pipe(WIN_WIDTH/2)
                                bird = Bird(10,240)
                                running = False
                                
                            elif pipe1.bottom <= (bird.y + bird.img_height):
                               
                                print('pipe1_bottom:', pipe1.bottom)
                                pipe1 = Pipe(WIN_WIDTH)
                                pipe2 = Pipe(WIN_WIDTH/2)
                                bird = Bird(10,240)
                                running = False
                            else:
                                score += 1
                                print(score)



                        if pipe2.x == bird.x + bird.img_width:
                            if pipe2.height >= bird.y:                           
                                print('pipe2_height:', pipe2.height)
                                pipe1 = Pipe(WIN_WIDTH)
                                pipe2 = Pipe(WIN_WIDTH/2)
                                bird = Bird(10,240)
                                running = False
                            elif pipe2.bottom <= (bird.y + bird.img_height):
                                print('pipe2_bottom:', pipe2.bottom)
                                pipe1 = Pipe(WIN_WIDTH)
                                pipe2 = Pipe(WIN_WIDTH/2)
                                bird = Bird(10,240)
                                running = False
                            else:
                                score += 1
                                print(score)


                        # 上下牆壁
                        if bird.y + bird.img_height == WIN_HEIGHT - 5 or bird.y - 10 <= 0:
                            pygame.time.delay(10)
                            bird = Bird(10,240)
                            running = False
                            print('touch padding')


                        # 水管加分
    #                     if game
                        # 微笑動作
                        if sm==1 and game_active:
                            bird.jump()


                        if running == False:
                            clock 

                head_font = pygame.font.Font('04B_19.ttf',40)                
                score_surface = head_font.render(f'Score: {int(score)}', True, (255, 255, 255))
    #             highscore_surface = head_font.render(f'HighScore: {int(high_score)}', True, (255, 255, 255))
                WIN.blit(score_surface, (80,100))  
    #             WIN.blit(highscore_surface, (80,50))





                pygame.display.update()

            print(score)

    #         if game_active:
    #             score_display('main_game')
    #         else:
    #             WIN.blit(game_over_surface,game_over_rect)
    #             high_score = update_score(score,high_score)
    #             score_display('game_over')


            stop = True
            while stop :

                for event in pygame.event.get():
#                     pygame.init()
                    # 建立 window 視窗畫布，大小為 800x600
#                     window_surface = pygame.display.set_mode((800, 600))
                    # 設置視窗標題為 Hello World:)
#                     pygame.display.set_caption('Hello World:)')
                    # 清除畫面並填滿背景色
                    WIN.fill((255, 255, 255))

                    # 宣告 font 文字物件
                    head_font = pygame.font.SysFont(None, 60)
                    # 渲染方法會回傳 surface 物件
                    text_surface = head_font.render(f'Score: {int(score)}', True, (0, 0, 0))
                    # blit 用來把其他元素渲染到另外一個 surface 上，這邊是 window 視窗
                    WIN.blit(text_surface, (10, 10))

                    # 更新畫面，等所有操作完成後一次更新（若沒更新，則元素不會出現）
                    running = True
                    pygame.display.update()
                    if event.type == QUIT:
                        pygame.quit()
                        sys.exit()

                    elif event.type == KEYDOWN:
                        if event.key == K_ESCAPE or event.key == K_q:
                            sys.exit(0)
                            camera.release()
                        if event.key == K_w:
                            start = True
                            running = True
                            stop = False
                            score = 0
                pygame.display.update()            
                            

            

                        


    except (KeyboardInterrupt, SystemExit):
        pygame.quit()
        cv2.destroyAllWindows()
        camera.release()

        
if __name__ == '__main__':    
    main()

In [ ]:
class Mosquito(pygame.sprite.Sprite):
    def init(self):
        super().init()
        pygame.sprite.Sprite.init(self)
        self.x = random.randrange(0, 640-IMAGEWIDTH)
        self.y = random.randrange(-20, 0)
        self.raw_image = pygame.image.load('UFO.png')
        self.image = pygame.transform.scale(self.raw_image, (IMAGEWIDTH, IMAGEHEIGHT))
        self.rect = self.image.get_rect()
        self.rect.topleft = (self.x, self.y)
        self.width = IMAGEWIDTH
        self.height = IMAGEHEIGHT
        self.widow_width = WINDOW_WIDTH
        self.window_height = WINDOW_HEIGHT

    def draw(self, win):
        win.blit(self.image, self.rect)
    def move(self):
        self.x += random.randrange(-10, 10)
        self.y += 10
        self.rect.topleft = (self.x, self.y)